In [1]:
import xlwings as xw
import pandas as pd
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']

In [13]:
import cx_Oracle
import os
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"

In [14]:
def DataWarehouse():
    try:
        os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
        cx_Oracle.init_oracle_client(lib_dir=LOCATION)
        connection = cx_Oracle.connect(
            user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

        )
        cursor = connection.cursor()
        return cursor

    except:

        connection = cx_Oracle.connect(
            user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

        )
        cursor = connection.cursor()
        return cursor

In [15]:
cur = DataWarehouse()

In [59]:
dic_dm = dict(cur.execute('select * from DELIVERY_METHOD').fetchall())

In [66]:
dict(zip(list(dic_dm.values()),list(dic_dm.keys())))

{'일반배송': 0, '택배': 1, '무진동차량': 2, '직배송': 3, 'ELSE': 99}

['일반배송', '택배', '무진동차량', '직배송', 'ELSE']

In [ ]:
inv_map = {v: k for k, v in map.items()}

In [18]:
import pandas as pd

In [19]:
df_dm= pd.DataFrame(cur.execute('select * from DELIVERY_METHOD').fetchall())

In [56]:
dict_dm = dict(zip(list(df_dm[1]),list(df_dm[0])))

In [57]:
dict_dm['일반배송']

0

In [70]:
def get_tb_idx(tb_name=str, content=str):
    """
    DW의 table이름을, content에는 테이블의 content를 입력하면 tb상 key값을 반환한다.
    """
    cur = DataWarehouse()
    dic_dm = dict(cur.execute(f'select * from {tb_name}').fetchall())
    dic_dm = dict(zip(list(dic_dm.values()),list(dic_dm.keys())))
    return dic_dm[content]

In [107]:
df_test = pd.read_csv('../test.csv',encoding='utf-8',index_col=0)

In [97]:
df_test['0'][0] = None

In [108]:
df_test.index=df_test.index+1

In [110]:
df_test.columns

Index(['0'], dtype='object')